In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os
import sys
import tqdm
# Modify the working path so that this.ipynb file can import other modules like in the root directory
current_dir = os.path.dirname(os.path.abspath('__file__'))
sys.path.append(os.path.join(current_dir, '..'))

In [2]:
dataset_names=["burgers", "kuramoto", "power", "ip"]
dataset_name=dataset_names[0] # can be changed

if dataset_name=="burgers":
    test_data_path=os.path.join("..", "datasets", "burgers_50_10_128.pkl")
elif dataset_name=="kuramoto":
    test_data_path=os.path.join("..", "datasets", "kuramoto_100_15_8_8.pkl")
elif dataset_name=="power":
    test_data_path=os.path.join("..", "datasets", "power_100_31_18_9.pkl")
elif dataset_name=="ip":
    test_data_path=os.path.join("..", "datasets", "inverted_pendulum_100_127_2_1.pkl")

test_data_dict=pickle.load(open(test_data_path,"rb"))
print(f"Dataset: {dataset_name}")
print("Test data loaded from: ", test_data_path)
Y_bar=test_data_dict['data']['Y_bar']
Y_f=test_data_dict['data']['Y_f']
U=test_data_dict['data']['U']
print("Y_bar shape: ", Y_bar.shape)
print("Y_f shape: ", Y_f.shape)
print("U shape: ", U.shape)

Dataset: burgers
Test data loaded from:  ..\datasets\burgers_50_10_128.pkl
Y_bar shape:  (50, 10, 128)
Y_f shape:  (50, 128)
U shape:  (50, 10, 128)


In [4]:

def natural_evolve(initial_state, target_state, dataset_name="burgers"):
    '''
    Return the final state vector given initial state vector without control input.
    '''
    s=initial_state # (state_dim,), the state vector (original meaning)

    if dataset_name=="burgers":
        from dynamics.burgers_dynamics import burgers_update
        for i in range(10): # Loop over time steps to update the state vector
            s=burgers_update(state=s,action=np.zeros(128),delta_t=0.1, t=None) # get the updated state vector (of original meaning)
    if dataset_name=="kuramoto":
        from dynamics.kuramoto_dynamics import kuramoto_update
        for i in range(15): # Loop over time steps to update the state vector
            s=kuramoto_update(state=s, action=np.zeros(8), delta_t=0.01, t=None) # get the updated state vector (of original meaning)
    if dataset_name=="power":
        from dynamics.swing_dynamics import swing_update
        for i in range(31): # Loop over time steps to update the state vector
            s=swing_update(state=s, action=np.zeros(9), delta_t=0.01, t=i*0.01) # get the updated state vector (of original meaning)
    if dataset_name=="ip":
        from dynamics.ip_dynamics import ip_update
        for i in range(127): # Loop over time steps to update the state vector
            s=ip_update(state=s, action=np.zeros(1), delta_t=0.01, t=None) # get the updated state vector (of original meaning)

    target_loss=((s-target_state)**2).mean() # MSE loss between target state and final state
    return s, target_loss # final state, (state_dim,); target loss, scalar; energy, scalar

def evaluate_natural_target_loss(Y_bar, Y_f, dataset_name):
    final_losses=[]
    n_trajectories=Y_bar.shape[0]
    for i in tqdm.tqdm(range(n_trajectories), desc='Looping over trajectories'):
        initial_state = Y_bar[i][0] # (state_dim,)
        target_state = Y_f[i] # (state_dim,)
        final_state, target_loss=natural_evolve(initial_state, target_state, dataset_name=dataset_name) # Natural evolution
        final_losses.append(target_loss)
    return np.mean(final_losses)

natural_target_loss = evaluate_natural_target_loss(Y_bar, Y_f, dataset_name=dataset_name)
print(f"Dataset: {dataset_name}")
print(f"Natural target loss: {natural_target_loss:.8f}")


Looping over trajectories: 100%|██████████| 50/50 [00:16<00:00,  3.06it/s]

Dataset: burgers
Natural target loss: 0.00353475


In [5]:
delta_t=0.1
energys=[]
for i in range(U.shape[0]):
    for t in range(U.shape[1]):
        energys.append((U[i,t]**2).sum()*delta_t)
print(np.array(energys).mean())

0.7154887737851607
